In [1]:
import random

In [2]:
class Card:
    """Represents a standard playing card."""

    suit_names = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    rank_names = [None, 'Ace', '2', '3', '4', '5', '6', '7', 
                  '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        rank_name = Card.rank_names[self.rank]
        suit_name = Card.suit_names[self.suit]
        return f'{rank_name} of {suit_name}'

    def __eq__(self, other):
        return self.suit == other.suit and self.rank == other.rank

    def to_tuple(self):
        return (self.suit, self.rank)

    def __lt__(self, other):
        return self.to_tuple() < other.to_tuple()

    def __le__(self, other):
        return self.to_tuple() <= other.to_tuple()

In [3]:
class Deck:

    def __init__(self, cards):
        self.cards = cards

    def __str__(self):
        res = []
        for card in self.cards:
            res.append(str(card))
        return '\n'.join(res)

    def take_card(self):
        return self.cards.pop()

    def put_card(self, card):
        self.cards.append(card)

    def shuffle(self):
        random.shuffle(self.cards)

    def sort(self):
        self.cards.sort()

    @staticmethod
    def make_cards():
        cards = []
        for suit in range(4):
            for rank in range(2, 15):
                card = Card(suit, rank)
                cards.append(card)
        return cards

In [4]:
class Hand(Deck):
    """Represents a hand of playing cards."""

    def __init__(self, label=''):
        self.label = label
        self.cards = []

    def move_cards(self, other, num):
        for i in range(num):
            card = self.take_card()
            other.put_card(card)

In [5]:
class BridgeHand(Hand):
    """Represents a bridge hand."""

    hcp_dict = {
        'Ace': 4,
        'King': 3,
        'Queen': 2,
        'Jack': 1,
    }

    def high_card_point_count(self):
        count = 0
        for card in self.cards:
            rank_name = Card.rank_names[card.rank]
            count += BridgeHand.hcp_dict.get(rank_name, 0)
        return count

In [6]:
def find_defining_class(obj, method_name):
    """Find the class where the given method is defined."""
    for typ in type(obj).mro():
        if method_name in vars(typ):
            return typ
    return f'Method {method_name} not found.'

hand = BridgeHand('player 3')
find_defining_class(hand, 'shuffle')

__main__.Deck

In [7]:
# 17.12.2. Exercise

class Trick(Deck):
    """Represents a trick in contract bridge."""

    def find_winner(self):
        led_suit = self.cards[0].suit

        following_suit = [(i, card) for i, card in enumerate(self.cards)
                          if card.suit == led_suit]
        
        winner_index, _ = max(following_suit, key=lambda x: x[1].rank)

        return winner_index

In [8]:
cards = [Card(1, 3),
         Card(1, 10),
         Card(1, 12),
         Card(2, 13)]
trick = Trick(cards)
print(trick)

3 of Diamonds
10 of Diamonds
Queen of Diamonds
King of Hearts


In [9]:
trick.find_winner()

2

In [18]:
# 17.12.3. Exercise

class PokerHand(Hand):
    """Represents a poker hand."""

    def __init__(self, cards=None, label=''):
        super().__init__(label)
        if cards is None:
            cards = []
        self.cards = cards

    def get_suit_counts(self):
        counter = {}
        for card in self.cards:
            key = card.suit
            counter[key] = counter.get(key, 0) + 1
        return counter
    
    def get_rank_counts(self):
        counter = {}
        for card in self.cards:
            key = card.rank
            counter[key] = counter.get(key, 0) + 1
        return counter    

    def has_flush(self):
        return any(count >= 5 for count in self.get_suit_counts().values())

    # 17.12.4. Exercise
    def has_straight(self):
        if len(self.cards) < 5:
            return False

        # Use a set of ranks to ignore duplicates
        ranks = {card.rank for card in self.cards}

        # Treat Ace (rank 14) as low as well by adding 1
        if 14 in ranks:
            ranks.add(1)

        for r in ranks:
            if all((r + offset) in ranks for offset in range(5)):
                return True

        return False

    # 17.12.4.
    def has_straight_flush(self):
        if not self.has_flush() or not self.has_straight():
            return False

            # Group cards by suit
        suit_groups = {}
        for card in self.cards:
            suit_groups.setdefault(card.suit, []).append(card)
    
        # For each suit, check if that suit's cards contain a straight
        for suit, cards in suit_groups.items():
            if len(cards) < 5:
                continue
    
            # Extract unique ranks for that suit
            ranks = {card.rank for card in cards}
            if 14 in ranks:
                ranks.add(1)  # Ace can be low
    
            # Look for 5 consecutive ranks in this suit
            for r in ranks:
                if all((r + offset) in ranks for offset in range(5)):
                    return True
        
        return False

    def has_pair(self):
        """Return True if the hand contains at least one pair."""
        rank_counts = self.get_rank_counts()
        # A pair means two or more cards of the same rank
        return any(count >= 2 for count in rank_counts.values())

    def has_full_house(self):
        """Return True if the hand contains a full house (three of one rank and two of another)."""
        rank_counts = self.get_rank_counts().values()
        has_three = any(count == 3 for count in rank_counts)
        has_two = any(count == 2 for count in rank_counts)
        return has_three and has_two

In [17]:
# 17.12.8. Exercise 

class Kangaroo:
    """A Kangaroo is a marsupial."""

    def __init__(self, name, contents=None):
        self.name = name
        if contents is None:
            contents = []
        self.contents = contents

    def __str__(self):
        t = [self.name + ' has pouch contents:']
        for obj in self.contents:
            s = '    ' + object.__str__(obj)
            t.append(s)
        return '\n'.join(t)

    def put_in_pouch(self, item):
        self.contents.append(item)

In [20]:
hand = PokerHand([
    Card(0, 12), Card(1, 12), Card(2, 12),  # three Queens
    Card(0, 9), Card(2, 9)                  # two Nines
])
print(hand.has_full_house())  # ✅ True


True
